In [8]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key='', 
    model_name="llama-3.3-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [53]:
import chromadb

client = chromadb.Client()
collection = client.create_collection(name="my_collection")

In [35]:
collection.add(
    documents=[
        "This is about New Delhi",
        "This is about New York"
    ],
    ids = ["id5","id6"],
    metadatas = [
        {"url" : "https://www.newdelhi.com/"},
        {"url" : "https://www.newyork.com/"}
    ]
)

In [36]:
docs=collection.get()
docs

{'ids': ['id1', 'id2', 'id3', 'id4', 'id5', 'id6'],
 'embeddings': None,
 'documents': ['This is about New Delhi',
  'This is about New York',
  'This document is about New York',
  'This document is about Delhi',
  'This is about New Delhi',
  'This is about New York'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [None,
  None,
  {'url': 'https://en.wikipedia.org/wiki/New_York_City'},
  {'url': 'https://en.wikipedia.org/wiki/New_Delhi'},
  {'url': 'https://www.newdelhi.com/'},
  {'url': 'https://www.newyork.com/'}]}

In [37]:
docs=collection.get(ids=["id1"])
docs

{'ids': ['id1'],
 'embeddings': None,
 'documents': ['This is about New Delhi'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [None]}

In [38]:
result = collection.query(
    query_texts=["I want to eat pani puri"],
    n_results=2
)
result

{'ids': [['id1', 'id5']],
 'embeddings': None,
 'documents': [['This is about New Delhi', 'This is about New Delhi']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None, {'url': 'https://www.newdelhi.com/'}]],
 'distances': [[1.478261113166809, 1.478261113166809]]}

In [41]:
collection.delete(ids="ids1")

In [44]:
collection.get()

{'ids': ['id1', 'id2', 'id3', 'id4', 'id5', 'id6'],
 'embeddings': None,
 'documents': ['This is about New Delhi',
  'This is about New York',
  'This document is about New York',
  'This document is about Delhi',
  'This is about New Delhi',
  'This is about New York'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [None,
  None,
  {'url': 'https://en.wikipedia.org/wiki/New_York_City'},
  {'url': 'https://en.wikipedia.org/wiki/New_Delhi'},
  {'url': 'https://www.newdelhi.com/'},
  {'url': 'https://www.newyork.com/'}]}

In [12]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.stepstone.de/stellenangebote--Product-Owner-Product-Data-Management-and-Presentation-m-f-d-Muenchen-Jochen-Schweizer-mydays-Holding-GmbH--12317502-inline.html?rltr=1_1_25_seorl_m_0_0_0_0_1_0")
page_data = loader.load().pop().page_content
print(type(page_data))

<class 'str'>


In [59]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.3-70b-versatile"
)

from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website which has the job posting details.
        Your job is to extract the job postings and return them in JSON format containing all the necessary information including the
        following keys: `role`, `experience`, `skills` , `employment type`, `location`  and `description` . Please add other keys if founds in the document and translate
        the text to english from any language received.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
res.content

'```json\n{\n  "role": "Product Owner: Product Data Management and Presentation",\n  "experience": "More than 3 years of experience in a Product role coming from a value- and customer-driven environment",\n  "skills": [\n    "Effective in communication, planning and good coordination skills",\n    "Good knowledge and proven experience in working with software development teams, preferably in agile frameworks like Scrum or Kanban",\n    "Fluent business communication in English. Ability to communicate in German is a plus but not required"\n  ],\n  "employment_type": "Full-time",\n  "location": "Munich, Germany",\n  "description": "Join our vibrant team at JSMD, a proud member of the ProSiebenSat.1 Group, where we bring people together through unforgettable real-life experiences. As our Product Owner: Product Data Management and Presentation, you will play a crucial role in enhancing our digital product offerings. Your ability to manage and present product data will help us deliver impac

In [60]:
res.content

'```json\n{\n  "role": "Product Owner: Product Data Management and Presentation",\n  "experience": "More than 3 years of experience in a Product role coming from a value- and customer-driven environment",\n  "skills": [\n    "Effective in communication, planning and good coordination skills",\n    "Good knowledge and proven experience in working with software development teams, preferably in agile frameworks like Scrum or Kanban",\n    "Fluent business communication in English. Ability to communicate in German is a plus but not required"\n  ],\n  "employment_type": "Full-time",\n  "location": "Munich, Germany",\n  "description": "Join our vibrant team at JSMD, a proud member of the ProSiebenSat.1 Group, where we bring people together through unforgettable real-life experiences. As our Product Owner: Product Data Management and Presentation, you will play a crucial role in enhancing our digital product offerings. Your ability to manage and present product data will help us deliver impac

In [61]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Product Owner: Product Data Management and Presentation',
 'experience': 'More than 3 years of experience in a Product role coming from a value- and customer-driven environment',
 'skills': ['Effective in communication, planning and good coordination skills',
  'Good knowledge and proven experience in working with software development teams, preferably in agile frameworks like Scrum or Kanban',
  'Fluent business communication in English. Ability to communicate in German is a plus but not required'],
 'employment_type': 'Full-time',
 'location': 'Munich, Germany',
 'description': 'Join our vibrant team at JSMD, a proud member of the ProSiebenSat.1 Group, where we bring people together through unforgettable real-life experiences. As our Product Owner: Product Data Management and Presentation, you will play a crucial role in enhancing our digital product offerings. Your ability to manage and present product data will help us deliver impactful experiences to our customers.',
 'r

In [63]:
cd 

 Volume in drive C is Acer
 Volume Serial Number is EE7D-8C3E

 Directory of C:\Users\indus\Documents\Python Project\Cover_letter_generator

04/06/2025  09:38 PM    <DIR>          .
04/04/2025  05:41 PM    <DIR>          ..
04/06/2025  03:24 PM             3,298 .gitignore
04/04/2025  06:24 PM    <DIR>          .ipynb_checkpoints
04/06/2025  06:28 PM    <DIR>          .vscode
04/06/2025  05:10 PM    <DIR>          apps
04/04/2025  06:01 PM    <DIR>          coverletter
04/06/2025  09:38 PM            34,864 groq_api.ipynb
04/06/2025  03:28 PM    <DIR>          imgs
04/06/2025  05:19 PM             1,610 readme.md
04/06/2025  05:19 PM               204 requirements.txt
04/06/2025  09:19 PM    <DIR>          vectorstore
               4 File(s)         39,976 bytes
               8 Dir(s)  143,560,527,872 bytes free


In [68]:
import pandas as pd

df = pd.read_csv("./apps/resources/my_resume.csv")
df

,Section,Details
0,Summary,With 6 years of experience as a Data Scientist...
1,SKILLS,"Programming Languages: Python, R, SQL, HTML/CS..."
2,PROFESSIONAL EXPERIENCE1,"Consultant Product Owner | Deutsche Bank, Fran..."
3,PROFESSIONAL EXPERIENCE2,"Data Scientist | Creditshelf AG, Frankfurt, Ge..."
4,PROFESSIONAL EXPERIENCE3,Data Scientist | FGFIS Global Services India L...
5,PROFESSIONAL EXPERIENCE4,Associate Consultant | Capgemini Technology Se...
6,PROFESSIONAL EXPERIENCE5,"Analyst | Axa Technology Services, Bengaluru, ..."
7,PROFESSIONAL EXPERIENCE6,"Junior Web Developer, Intern | Treat Well, Ben..."
8,ACADEMIC EDUCATION,Bachelor of Engineering – Electronics and Comm...
9,ACTIVITIES,"Certified Open Water Diver, Padi Nov 2024\nAwa..."


In [69]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Section"],
                       metadatas={"details": row["Details"]},
                       ids=[str(uuid.uuid4())])

In [70]:
json_res.items()

dict_items([('role', 'Product Owner: Product Data Management and Presentation'), ('experience', 'More than 3 years of experience in a Product role coming from a value- and customer-driven environment'), ('skills', ['Effective in communication, planning and good coordination skills', 'Good knowledge and proven experience in working with software development teams, preferably in agile frameworks like Scrum or Kanban', 'Fluent business communication in English. Ability to communicate in German is a plus but not required']), ('employment_type', 'Full-time'), ('location', 'Munich, Germany'), ('description', 'Join our vibrant team at JSMD, a proud member of the ProSiebenSat.1 Group, where we bring people together through unforgettable real-life experiences. As our Product Owner: Product Data Management and Presentation, you will play a crucial role in enhancing our digital product offerings. Your ability to manage and present product data will help us deliver impactful experiences to our cus

In [75]:
result_list=[]
for key, value in json_res.items():
    value = collection.query(query_texts=key, n_results=2).get('metadatas', [])
    result_list.append(value)

In [76]:
result_list

[[[{'details': 'Programming Languages: Python, R, SQL, HTML/CSS\nReporting Tools: Tableau, Power BI, SAS Visual Analytics\nData Warehousing Tools: MySQL, Oracle DB, PostgreSQL, AWS S3, Google BigQuery, Apache Spark, Azure Databricks, AWS Athena, Apache Kafka, Dremio\nStatistics and ML: CART, NLP, Clustering, Sentiment Analysis, Deep Learning, Time Series Analysis, AI, Langchain, Linear/Logistic Regression, Classification\nCloud Environment: Amazon Web Services, Google Cloud Platform, Docker, Kubernets, Git\nLibraries: TensorFlow, Keras, Spark MLlib, NLTK, Scikit-Learn, SciPy, Pandas, NumPy, Seaborn, TensorFlow, Plotly, Texthero'},
   {'details': 'Associate Consultant | Capgemini Technology Services Pvt. Ltd, Bengaluru, IN | Dec ’16 – Oct’ 19\n1. Upskilling and Digital Transformation\n•Developed the Decision Tree to find the most suitable training and send emailers to the employees to take the most relevant training for upskilling.\n•Build and construct prototypes, proof of concepts and

In [179]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are XXX, a Data scientist with 8 years of industry experience in different domains of work. 
        You are applying for the job with the {job_description}.
        Your job is to write an email to the recruiting manager of the job mentioned above describing your capability of fulfilling 
        their needs and your motivation to work fro the company. You can also give examples from the experience before to make it more 
        natural and effective. Also add the most relevant skill from the following skills to showcase your portfolio: {link_list}
        Remember you are XXX, a data scientist.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(json_res), "link_list": result_list[0] })
print(res.content)

Subject: Application for Product Owner: Product Data Management and Presentation Role

Dear Hiring Manager,

I am excited to apply for the Product Owner: Product Data Management and Presentation role at Jochen Schweizer mydays Holding GmbH, as advertised. With over 8 years of experience in the industry as a Data Scientist, I am confident that my skills and expertise make me an ideal candidate for this position.

As a seasoned data professional, I have a strong background in managing and presenting complex data to drive business decisions. My experience in working with cross-functional teams, including software development teams, has equipped me with the skills to effectively communicate and collaborate with stakeholders to deliver impactful results. I am well-versed in agile frameworks like Scrum and Kanban, which I believe will enable me to make a seamless transition into this role.

In my previous role as an Associate Consultant at Capgemini Technology Services, I developed and imple